In [ ]:
import sounddevice as sd
import numpy as np
import queue
import threading
from faster_whisper import WhisperModel

c:\Users\GauravAmbartani\miniconda3\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:

samplerate = 16000
block_duration = 0.5  # seconds
chunk_duration = 2    # seconds
channels = 1
frames_per_block = int(samplerate * block_duration)
frames_per_chunk = int(samplerate * chunk_duration)
audio_queue = queue.Queue()
audio_buffer = []

# Model setup
#model = WhisperModel("large-v3", device="cpu", compute_type="int8")

#model = WhisperModel("small", device="cpu", compute_type="int8")

model = WhisperModel("base", device="cpu", compute_type="int8")


def audio_callback(indata, frames, time, status):
    if status:
        print(status)
    audio_queue.put(indata.copy())

def recorder():
    with sd.InputStream(samplerate=samplerate, channels=channels, callback=audio_callback, blocksize=frames_per_block):
        print("Listening... Press Ctrl+C to stop.")
        while True:
            sd.sleep(100)

def transcriber():
    global audio_buffer
    try:
        while True:
            block = audio_queue.get()
            audio_buffer.append(block)
            total_frames = sum(len(b) for b in audio_buffer)
            if total_frames >= frames_per_chunk:
                audio_data = np.concatenate(audio_buffer)[:frames_per_chunk]
                audio_buffer = []  # Clear buffer
                audio_data = audio_data.flatten().astype(np.float32)

                # Transcription
                segments, _ = model.transcribe(
                    audio_data,
                    language="en",
                    beam_size=1
                )

                for segment in segments:
                    print(f"{segment.text}")
    except KeyboardInterrupt:
        print("\nTranscription stopped by user.")

# Start recorder thread and run transcriber
threading.Thread(target=recorder, daemon=True).start()
transcriber()






Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`
c:\Users\GauravAmbartani\miniconda3\Lib\site-packages\huggingface_hub\file_download.py:143: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\GauravAmbartani\.cache\huggingface\hub\models--Systran--faster-whisper-base. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode,

Listening... Press Ctrl+C to stop.
input overflow
 Oh, is it?


: 